In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Bidirectional, Conv1D, Dense, LSTM, Flatten
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

trnX = np.load("trnX.npy")
trnY = np.load("trnY.npy")
valX = np.load("valX.npy")
valY = np.load("valY.npy")
tstX = np.load("tstX.npy")

mu = trnX.mean()
sigma = trnX.std()
trnX = (trnX - mu) / sigma
valX = (valX - mu) / sigma
tstX = (tstX - mu) / sigma

model = Sequential()
model.add(Conv1D(64, 2, padding = "same", activation = "relu", input_shape = trnX.shape[1:]))
model.add(Conv1D(64, 2, padding = "same", activation = "relu"))
model.add(Bidirectional(LSTM(64, dropout = 0.2, recurrent_dropout = 0.2)))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

callbacks = [ ReduceLROnPlateau(monitor = "val_acc", patience = 5), EarlyStopping(monitor = "val_acc", patience = 10, restore_best_weights = True) ]
history = model.fit(trnX, trnY, batch_size = 32, epochs = 100, validation_data = (valX, valY), callbacks = callbacks)

probabilities = model.predict(tstX)

predictions = open("predictions.csv", "w")
predictions.write("id,label\n")
for i in range(tstX.shape[0]):
    if (probabilities[i,0] > 0.5):
        predictions.write(str(i).zfill(4) + ",1\n")
    else:
        predictions.write(str(i).zfill(4) + ",0\n")
predictions.close()